In [8]:
import os
from dataclasses import dataclass

import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image
from datasets import load_metric
from torch.utils.data import Dataset
from transformers import (
    VisionEncoderDecoderModel,
    TrOCRProcessor,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    default_data_collator
)

from ultralytics import YOLO

In [2]:
def seed_everything(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [36]:
# Инициализируем распознавание текста
processor = TrOCRProcessor.from_pretrained("raxtemur/trocr-base-ru")
model = VisionEncoderDecoderModel.from_pretrained("../../models/text_recognizer", local_files_only=True)
model.to(device)

SafetensorError: Error while deserializing header: MetadataIncompleteBuffer

In [10]:
text_detector = YOLO("../../models/new_text_detector/best.pt").to(device)

In [13]:
import pathlib

def get_rand_image():
    path = pathlib.Path("../../data/raw/Распознавание текстов/Уставные грамоты в jpg (Просветов)")
    return np.random.choice(list(path.iterdir()))

In [35]:
from PIL import ImageOps

# img = Image.open(r"../../data/raw/Распознавание текстов/Уставные грамоты в jpg (Просветов)/11227024_doc1.jpg")
img = Image.open(get_rand_image())
img = ImageOps.exif_transpose(img)

res = text_detector.predict([img])

# Process results list
for result in res:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    result.show()  # display to screen

0: 256x192 12 texts, 131.0ms
Speed: 2.0ms preprocess, 131.0ms inference, 6.0ms postprocess per image at shape (1, 3, 256, 192)
